In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

train = pd.read_csv("../data/train.csv")
train_labels = train["Label"].values
train_data = train.drop(labels = ["Label", "Img_Name"], axis = 1).values

raw_train_data, raw_test_data, raw_train_labels, raw_test_labels = train_test_split(train_data, train_labels, test_size = 0.20)

#The images consist of grayscale values between 0.0 and 255.0. We normalize them by dividing the whole data arrays by 255.0.
# normalizing

X_train_data = np.asarray(raw_train_data / 255.0, dtype=float)
X_test_data = np.asarray(raw_test_data / 255.0, dtype=float)
Y_train_labels= raw_train_labels # np.asarray(raw_test_labels , dtype=np.int32)
Y_test_labels = raw_test_labels # np.asarray(raw_train_labels , dtype=np.int32)

# print 'X_train_data' + str(raw_test_data.head())
# print 'X_test_data' + str(X_test_data.head())
# print 'Y_train_labels' + str(Y_train_labels.head())
# print 'Y_test_labels' + str(Y_test_labels.head())

print('X_train_data.size=', len(X_train_data))
print('X_test_data.size=', len(X_test_data))
print('Y_train_labels.size=', len(Y_train_labels))
print('Y_test_labels.size=', len(Y_test_labels))

Using TensorFlow backend.


('X_train_data.size=', 374)
('X_test_data.size=', 94)
('Y_train_labels.size=', 374)
('Y_test_labels.size=', 94)


In [4]:
nets = 3
model = [0] *nets

for j in range(3):
    model[j] = Sequential()
    model[j].add(Conv2D(24,kernel_size=5,padding='same',activation='relu', input_shape=(32, 32, 3)))
    model[j].add(MaxPool2D())
    if j>0:
        model[j].add(Conv2D(48,kernel_size=5,padding='same',activation='relu'))
        model[j].add(MaxPool2D())
    if j>1:
        model[j].add(Conv2D(64,kernel_size=5,padding='same',activation='relu'))
        model[j].add(MaxPool2D(padding='same'))
    model[j].add(Flatten())
    model[j].add(Dense(256, activation='relu'))
    model[j].add(Dense(10, activation='softmax'))
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    

In [5]:
for j in range(3):
    model[j].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 24)        1824      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 24)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6144)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               1573120   
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total params: 1,577,514
Trainable params: 1,577,514
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


In [6]:

# TRAIN NETWORKS
history = [0] * nets
names = ["(C-P)x1","(C-P)x2","(C-P)x3"]
epochs = 20
for j in range(nets):
    history[j] = model[j].fit(X_train_data, Y_train_labels, batch_size=10, epochs=20, 
                              validation_data = (X_test_data,Y_test_labels), verbose=0)
    
    print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        names[j],epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

ValueError: Error when checking input: expected conv2d_3_input to have 4 dimensions, but got array with shape (374, 4)